In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

# Clear session
tf.keras.backend.clear_session()

# Turn off TensorFlow warning messages in program output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load the training data
training_data_df = pd.read_csv('Data/sales_data_training.csv', dtype=float) # expect float-point numbers

# Get X and Y data
X_training = training_data_df.drop('total_earnings', axis=1).values # call .values to get back the result as an array
Y_training = training_data_df[['total_earnings']].values

# Load testing data
test_data_df = pd.read_csv('Data/sales_data_test.csv', dtype=float) # expect float-point numbers

# Get X and y test data
X_testing = test_data_df.drop('total_earnings', axis=1).values # .values for an array
Y_testing = test_data_df[['total_earnings']].values

''' 
  Scale the data (normalizing it to a range)
  
  The scaler scales the data by multiplying it by a constant number and adding a constant number.
'''

# Preprocess data
X_scaler = MinMaxScaler(feature_range=(0, 1))
Y_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the data using scaler
X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)

# Scale testing
X_scaled_testing = X_scaler.transform(X_testing)
Y_scaled_testing = Y_scaler.transform(Y_testing)

# Print shape
print(X_scaled_testing.shape)
print(Y_scaled_testing.shape)

print('Note: Y values were scaled by multiplying by {:.10f} and adding {:4f}'.format(Y_scaler.scale_[0], Y_scaler.min_[0]))

################################################
####### Load up trained model checkpoint
################################################

# Initialize a session so that we can run TensorFlow operations
with tf.Session() as session:
     
    #####################################################################    
    # When loading from a checkpoint, don't initialize the variables!
    # session.run(tf.global_variables_initializer())
    
    #  It's important that when loading from a checkpoint, you don't call tf.global_variables_initializer. 
    #  Since all the variable values are loaded from a file, we don't want to initialize them back 
    #  to their default values. 
    #####################################################################

    # Load checkpoint from disk
    saver.restore(session, 'logs/trained_model.ckpt')
    print('Trained model successfully loaded from disk.')
    
    # Get the first accuracy score by running the 'cost' operation
    training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: Y_scaled_training})
    testing_cost = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_scaled_testing})
    
    print('Final Training Cost: {}'.format(training_cost))
    print('Final Testing Cost: {}'.format(testing_cost))
    
    # Predict
    Y_predicted_scaled = session.run(prediction, feed_dict={X: X_scaled_testing})
    
    # Unscale the data back to its original units
    Y_predicted = Y_scaler.inverse_transform(Y_predicted_scaled)
    real_earnings = test_data_df['total_earnings'].values[0]
    predicted_earnings = Y_predicted[0][0]

    print("The actual earnings of Game #1 were ${}".format(real_earnings))
    print("Our neural network predicted earnings of ${}".format(predicted_earnings))